<a href="https://colab.research.google.com/github/melirandazzo/trabajo_ciencia_datos/blob/main/ejercicio_clase_21_dea_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



En esta oportunidad nos reuniremos en grupos de máximo 4 personas.

Elegir 4 variables independientes que consideren útiles para predecir los “costos” de nuevos clientes [https://www.kaggle.com/datasets/mirichoi0218/insurance] <br>

Realizar el Encoding de las variables independientes (una persona hace el código y comparte, los demás ayudan dando instrucciones, etc) para generar matriz para le modelo
Elegir uno de los modelos aprendidos en clase y entrenarlo
Generar una predicción de costos sobre uno de los estudiantes del grupo


In [ ]:
#Importacion ded librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# descargando el dataset de kaggle: tres puntitos copy api comand
! kaggle datasets download -d mirichoi0218/insurance

Dataset URL: https://www.kaggle.com/datasets/mirichoi0218/insurance
License(s): DbCL-1.0
  0% 0.00/16.0k [00:00<?, ?B/s]
100% 16.0k/16.0k [00:00<00:00, 28.6MB/s]


In [ ]:
! unzip insurance.zip

Archive:  insurance.zip
  inflating: insurance.csv           


In [ ]:
#Importacion de los datos
dataset = pd.read_csv("/content/insurance.csv", sep = ",")
dataset

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


### EDA

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [ ]:
dataset.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [ ]:
dataset.describe(include="object")

,sex,smoker,region
count,1338,1338,1338
unique,2,2,4
top,male,no,southeast
freq,676,1064,364


# Data wrangling

#### varaibles categoricas

In [ ]:
categoricas = dataset.select_dtypes(include="object")
categoricas.head(2)

# sabemos que estas tres cataegorias son categoricas no ordinales.

,sex,smoker,region
0,female,yes,southwest
1,male,no,southeast


In [ ]:
# importo label encoder
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

In [ ]:
categoricas.columns
categoricas_le = categoricas.apply(LE.fit_transform)
categoricas_le

# 0 female - 1 male | 0 yes - 1 no | 0 northeast - 1 northwest - 2 southeast - 3 soutwest

,sex,smoker,region
0,0,1,3
1,1,0,2
2,1,0,2
3,1,0,1
4,1,0,1
...,...,...,...
1333,1,0,1
1334,0,0,0
1335,0,0,2
1336,0,0,3


#### numericas, escalar

probar sin esto

In [ ]:
numericas = dataset.select_dtypes(exclude="object")
numericas.columns

Index(['age', 'bmi', 'children', 'charges'], dtype='object')

In [ ]:
from sklearn.preprocessing import StandardScaler
SS = StandardScaler()

In [ ]:
numericas_escaladas = SS.fit_transform(numericas)
numericas_escaladas

array([[-1.43876426, -0.45332   , -0.90861367,  0.2985838 ],
       [-1.50996545,  0.5096211 , -0.07876719, -0.95368917],
       [-0.79795355,  0.38330685,  1.58092576, -0.72867467],
       ...,
       [-1.50996545,  1.0148781 , -0.90861367, -0.96159623],
       [-1.29636188, -0.79781341, -0.90861367, -0.93036151],
       [ 1.55168573, -0.26138796, -0.90861367,  1.31105347]])

#### merge categoricas y escaladas

In [ ]:
df = numericas.merge(categoricas_le, left_index=True, right_index=True)
df.head(3)

,age,bmi,children,charges,sex,smoker,region
0,19,27.90,0,16884.9240,0,1,3
1,18,33.77,1,1725.5523,1,0,2
2,28,33.00,3,4449.4620,1,0,2


In [ ]:
df.info()
df["sex"] = df["sex"].astype("category")
df["smoker"] = df["smoker"].astype("category")
df["region"] = df["region"].astype("category")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   bmi       1338 non-null   float64
 2   children  1338 non-null   int64  
 3   charges   1338 non-null   float64
 4   sex       1338 non-null   int64  
 5   smoker    1338 non-null   int64  
 6   region    1338 non-null   int64  
dtypes: float64(2), int64(5)
memory usage: 73.3 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   age       1338 non-null   int64   
 1   bmi       1338 non-null   float64 
 2   children  1338 non-null   int64   
 3   charges   1338 non-null   float64 
 4   sex       1338 non-null   category
 5   smoker    1338 non-null   category
 6   region    1338 non-null   category
dtypes: category(3), float

## Modelo

```
# Tiene formato de código
```



In [ ]:


X = df.drop("charges", axis=1) # Variables independientes: age, bmi, children, sex, smoker, region
y = df["charges"] # Variable objetivo: charges

# 2) Separacion en train y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



### Linear Regression

In [ ]:
#Entrenamiento del modelo
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [ ]:
regressor.coef_ # en orden:  age, bmi, children, sex, smoker, region
regressor.intercept_
X.columns
coeff_df = pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])
coeff_df

,Coefficient
age,257.056264
bmi,335.781491
children,425.091456
sex,-18.791457
smoker,23647.818096
region,-271.284266


In [ ]:
#Realizando las predicciones
y_pred = regressor.predict(X_test)
y_pred

array([ 8924.40724442,  7116.29501758, 36909.01352144,  9507.87469118,
       27013.3500079 , 10790.77956153,   226.29844571, 16942.71599941,
        1056.63079407, 11267.91997309, 28048.59793155,  9424.36324087,
        5326.32232088, 38460.06017922, 40303.40597026, 37147.01010262,
       15287.91876684, 35965.05485917,  9179.1753067 , 31510.8319849 ,
        3797.79068365, 10070.82803304,  2312.57551348,  7074.41348194,
       11352.37224357, 12907.77079523, 14448.84678727,  6205.65997921,
        9917.00839638,  2239.50032819,  9060.55469043, 13120.56214535,
        4617.70702822,  3467.91218926,  4402.74821855, 12967.91608907,
        1927.44498944,  8757.9180081 , 33324.35180597, 32638.47697026,
        3852.41756615,  4370.39670883, 14080.76023234, 11478.63402576,
        8829.26135924, 12046.15119133,  5322.80515731,  3100.71182484,
       35546.60547574,  9201.61196817, 15894.23763341,  2406.04003607,
       12397.52052544,  1433.90617387, 13448.14094304, 12519.54174599,
      

In [ ]:
y_test

,charges
764,9095.06825
887,5272.17580
890,29330.98315
1293,9301.89355
259,33750.29180
...,...
109,47055.53210
575,12222.89830
535,6067.12675
543,63770.42801


In [ ]:
df_predicciones = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df_predicciones['Sesgo']=df_predicciones.Actual -df_predicciones.Predicted
df_predicciones['Error_porc']=((df_predicciones.Actual -df_predicciones.Predicted)/df_predicciones.Actual) *100
df_predicciones

,Actual,Predicted,Sesgo,Error_porc
764,9095.06825,8924.407244,170.661006,1.876413
887,5272.17580,7116.295018,-1844.119218,-34.978333
890,29330.98315,36909.013521,-7578.030371,-25.836264
1293,9301.89355,9507.874691,-205.981141,-2.214400
259,33750.29180,27013.350008,6736.941792,19.961136
...,...,...,...,...
109,47055.53210,39116.968669,7938.563431,16.870627
575,12222.89830,11814.555568,408.342732,3.340801
535,6067.12675,7638.107736,-1570.980986,-25.893327
543,63770.42801,40959.081722,22811.346288,35.771042


In [ ]:
df.columns

Index(['age', 'bmi', 'children', 'charges', 'sex', 'smoker', 'region'], dtype='object')

#### 4) Prediccion sobre un estudiante


In [ ]:
# input de datos del nuevo caso
nombre = input("Ingrese su nombre: ")
edad = int(input("Ingrese su edad en años: "))
imc = float(input("Ingrese su indice de masa corporal: "))
hijos = int(input("Ingrese el numero de hijos que tiene: "))
sexo =  int(input("Ingrese 0 si es femenino, 1 si es masculino: "))
fuma = int(input("Ingrese 0 si fuma, 1 si no fuma: "))
region = int(input("Ingrese el numero de su region (0 northeast - 1 northwest - 2 southeast - 3 soutwest): "))

# cree un dataframe con mis datos, region pongo cuaquiera, 0

X1 = {'age':edad,'bmi':imc,'children':hijos,'sex':sexo,'smoker':fuma ,'region':region}
X1 = pd.DataFrame(X1, index=[0])
X1

#Realizando las predicciones
y_pred_X1 = regressor.predict(X1)
y_pred_X1
print("El costo en salud de ", nombre, "sería: ", y_pred_X1.round(2), "pesos")

Ingrese su nombre: Luisa
Ingrese su edad en años: 28
Ingrese su indice de masa corporal: 20
Ingrese el numero de hijos que tiene: 0
Ingrese 0 si es femenino, 1 si es masculino: 0
Ingrese 0 si fuma, 1 si no fuma: 0
Ingrese el numero de su region (0 northeast - 1 northwest - 2 southeast - 3 soutwest): 0
El costo en salud de  Luisa sería:  [1966.6] pesos
